In [ ]:
import os
import yaml
import numpy as np
import pandas as pd
from PIL import Image
import io

import tensorflow as tf
from tensorflow import keras

from src.model_creation import predict
from src.model_creation import neural_net

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "bigbucket"  # @param {type:"string"}
from datetime import datetime
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse
!mkdir -p mush
!mkdir -p inat
bucket_name='medium_mush'
bucket_name2='inatdatabase'
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name} mush
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name2} inat

In [ ]:
with open('model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
optimizer = keras.optimizers.RMSprop(
    lr=config["INITIAL_LEARNING_RATE"],
    rho=config["RMSPROP_RHO"],
    momentum=config["RMSPROP_MOMENTUM"],
    epsilon=config["RMSPROP_EPSILON"]
)

In [ ]:
model = make_neural_network(
    base_arch_name = "effecientnetv2l",
    weights = config['PRETRAINED_MODEL'],
    image_size = config['IMAGE_SIZE'],
    dropout_pct = config['DROPOUT_PCT'],
    n_classes = config['NUM_CLASSES'],
    input_dtype = tf.float32,
    train_full_network = False)

keras.models.load_model(config['FINAL_SAVE_DIR'])

model.compile(
    loss=config['LOSS'],
    optimizer=optimizer,
    metrics=[
    "accuracy", 
    tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top3 accuracy"),
    tf.keras.metrics.TopKCategoricalAccuracy(k=10, name="top10 accuracy")
]
)

In [ ]:
probs, max_prob = predict.make_prediction('file_path', config['IMAGE_SIZE'])